In [26]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 0 ns (started: 2022-09-22 01:44:45 -05:00)


# MODULES

In [27]:
import panel as pn
import panel.widgets as pnw
pn.extension('plotly')

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import numpy as np
import math

import pandas as pd

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
from scipy.spatial import distance

time: 15 ms (started: 2022-09-22 01:44:48 -05:00)


# CLASESS

## Vec2d

In [28]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

time: 0 ns (started: 2022-09-22 01:44:57 -05:00)


## Explorers

In [48]:
class Explorers(object):
    
    # This class substitutes BM trajectory and CREW Trajectory
    def __init__(self, x = 0, y = 0):
        self.x = x
        self.y = y
        
    def trace_trajectories(self, n_traj = 1, traj_type= "BM_2d", n_steps =1000, speeds = [6], crw_exponents = [0.5]):
        """
            Generates the trajectories of the explorers according to their type
            Arguments: 
                traj_type:
                n_steps:
                speed:
                n_traj:
                crw_exponents:
          """
        
        trajectories_df = pd.DataFrame(columns=['x','y','traj'])
        

        for j in range(n_traj):
            
            velocity = Vec2d(speeds[j], 0)
            
            
            traj_aux_array = np.ones((n_steps,3))*[self.x,self.y,0]
            traj_aux_array[0,2] = j

            for i in range(1,n_steps):
                crw_ex = lambda : crw_exponents[j] if len(crw_exponents) > 1 else i
                turn_angle = self.generate_angle(traj_type, crw_ex())
                
                velocity = velocity.rotated(turn_angle)

                traj_aux_array[i] = traj_aux_array[i-1,:]+[velocity.x,velocity.y,0]
                traj_aux_array[i,2] = j

                
            temp_df = pd.DataFrame(data = traj_aux_array, columns=['x','y','traj'])
            
            trajectories_df = pd.concat([trajectories_df,temp_df], ignore_index=True)
          
        return trajectories_df
    
    
    
    
    # angle choice 
    def generate_angle(self,traj_type, exponent = 0.6):
            
        if traj_type == "BM_2d":
            angle = np.random.uniform(low=-np.pi, high=np.pi)
        
        if traj_type == "BM_nsew":
            angle = np.random.choice([0,np.pi/2, np.pi, 3*np.pi])
        
        if traj_type == "CRW":
            angle = wrapcauchy.rvs(exponent)
            
            
        return angle

time: 16 ms (started: 2022-09-22 02:31:15 -05:00)


## Levy Flight

In [30]:
class LFlyTrajectory(object):
    
    def __init__(self, x,y):
        self.x=0
        self.y=0
        
        
    def levy_walk(self, alpha = 1, beta = 1, loc = 6 , speed = 3, samples = 100000):
        # Init velocity vector

        velocity = Vec2d(speed,0)

        # Init df
        LW_df = pd.DataFrame(columns=['x','y','traj'])
        lw_3d = np.array([[0,0,0]])
        aux = np.array([[0,0,0]])


        i = 1
        while i < samples:
            # get random n_steps form levy distribution
            step_size = levy_stable.rvs(alpha, beta, loc)
            step_size = int(np.ceil(abs(step_size)))

            theta = wrapcauchy.rvs(c=0.7, loc=0)

            # update velocity
            velocity = velocity.rotated(theta)

            for j in range(step_size):
                aux[0,:] = lw_3d[i-1,:]+[velocity.x,velocity.y,0]
                lw_3d = np.r_[lw_3d,aux]

                i+=1

        temp_df = pd.DataFrame(data = lw_3d, columns=['x','y','traj'])
        LW_df = pd.concat([LW_df,temp_df], ignore_index=True)

        return LW_df;

time: 0 ns (started: 2022-09-22 01:45:07 -05:00)


## Metrics

### Path lenght

In [38]:
class TrajectoryMetrics():
    
    
    def get_path_lengt(self, traj_df):

        euc_dis = np.array([get_euclidean_distance(traj_df.iloc[i-1], traj_df.iloc[i]) for i in range(1,traj_df.shape[0])])
        pl_df = pd.DataFrame(data = np.cumsum(euc_dis), columns = ['distance'])

        return pl_df
    
    
    
    def get_msd(self, traj_df):
        
        msd = np.empty(shape=(0))
        
        for tau in range(1,traj_df.shape[0]):
            BM_msd = get_msd(tau,traj_df)

            msd = np.append(msd,BM_msd)
        
        return msd
    
    
    
    def get_ta_distribution(self, traj_df):
        
        ta_dist = np.empty(shape=(0))

    
        for index, row in traj_df[1:-1].iterrows():
            
            turning_angle = get_turning_angle(traj_df.iloc[index-1],traj_df.iloc[index],traj_df.iloc[index+1])
            ta_dist = np.append(ta_dist,turning_angle)
        
        return ta_dist
    
    
    
    def get_sl_distribution(self, traj_df):
        
            ta = self.get_ta_distribution(traj_df)
            
            ta_df= pd.DataFrame(data = ta, columns=['TA_Levy'])
            
            
            sl_array = np.empty(shape=(0))
            
            for group, elements in ta_df.groupby((ta_df['TA_Levy'].shift()!= ta_df['TA_Levy']).cumsum()):
                if elements.shape[0]>1:
                    sl_array = np.append(sl_array,elements.shape[0]+1)
                    
            return sl_array

time: 0 ns (started: 2022-09-22 02:22:29 -05:00)


## Auxiliar functions

### Euclidean distance

$d_E(p,q)=\sqrt{(p_x-q_x)^2+(p_y-q_y)^2}$

In [31]:
def get_euclidean_distance(p,q):
    """
        Arguments:
            p: [x,y] values for the starting point
            q: [x,y] values for the ending point
    """
  
    distance = np.sqrt(np.square(p[0]-q[0]) + np.square(p[1]-q[1]))

    return distance

time: 0 ns (started: 2022-09-22 01:45:10 -05:00)


### Mean Square Displacement


$MSD = \frac{1}{N-n} \sum \limits_{i=1}^{N-n}(\vec{r}_{i+n}-\vec{r}_i)^2 \quad\quad n=1,...,N-1$
<br><br>

$MSD = \frac{1}{N-n}\sum \limits_{i=1}^{N-n}{d_E(p,q)}^2 \quad\quad n=1,...,N-1$

In [32]:
def get_msd(tau,path):
    """
      Arguments:
        tau:
        path:
    """

    square_displacement = 0 

    for i in range(tau,path.shape[0],1):
        square_displacement += np.square(get_euclidean_distance(path.iloc[i-tau], path.iloc[i]))

    msd= (1/(path.shape[0]-tau))*square_displacement

    return msd

time: 0 ns (started: 2022-09-22 01:45:12 -05:00)


### Turning Angle

$
tan(\phi)=\frac{|\vec{p}\times\vec{q}|}{\vec{p}\cdot\vec{q}}
$

In [33]:
def get_turning_angle(a,b,c,round_to_zero = False):
    """
      Arguments:
        a: coordinates for p vector's tail
        b: coordinates p vector's head / q vector's tail
        c: coordinates q vector's head
        round_to_zero: if true checks if value is close enough to zero to be take as it
     """

    p = np.subtract(b,a)
    q = np.subtract(c,b)

  
    pq_cross = np.cross(p,q)
    pq_scalar = np.dot(p,q)

    phi_angle = np.arctan2(pq_cross,pq_scalar)

    if round_to_zero:
        if  count_like_zero(phi_angle):
            phi_angle = 0
      
    return phi_angle

time: 0 ns (started: 2022-09-22 01:45:14 -05:00)


### Custom round

In [ ]:
def count_like_zero(number):
    """
        Arguments:
            number: 
        Return: True if -0.009 <= number <= 0.009 False otherwise
    """
    is_zero = False

    if  ((number >= -0.009) & (number <= 0)) | ((number >= 0) & (number <= 0.009)):
        is_zero = True

    return is_zero

### Is Negligible turn

In [34]:
def drop_turn(angle_i, angle_j):
    """
      Arguments:
        angle_i:
        angle_j: 
      Return: True if difference between angles is less than 0.001 False otherwise
      """
    is_dropable = False
    
    if (angle_i > 0) & (angle_j > 0): 
        rest = abs(angle_i - angle_j)
        if abs(angle_i - angle_j) < 0.001:
            is_dropable = True

    return is_dropable

time: 0 ns (started: 2022-09-22 01:45:16 -05:00)


# PLOT FUNCTIONS

## Plot Trajectories

In [35]:
def plot_trajectories(trajectories_df, n_trajectories = 1, title = "", line_name_prefix="", line_name_subfix = [""]):
    
    fig_3d = go.Figure()

    for i in range(n_trajectories):
        
        subfix = lambda : line_name_subfix[i] if len(line_name_subfix) > 1 else i
        trajectory_name = line_name_prefix +": " + str(subfix())

        fig_3d.add_scatter3d(
                x = trajectories_df.loc[trajectories_df['traj']==i,'x'], 
                y = trajectories_df.loc[trajectories_df['traj']==i,'y'], 
                z = trajectories_df.index,
                marker = dict(size=2),
                line = dict(width=2),
                mode = 'lines',
                name = trajectory_name,
                showlegend = True)

    
    fig_3d.update_layout(
            title_text = title,
            autosize = False,
            width = 800,
            height = 800,
            scene_camera = dict(eye = dict(x=0, y=0, z=2.5)),
            scene = dict(
                xaxis = dict(title = 'x_pos (mm'),
                yaxis = dict(title = 'y_pos (mm)'),
                zaxis = dict(title = 'time', nticks = 20)
            ))

    fig_3d.show()

time: 0 ns (started: 2022-09-22 01:45:18 -05:00)


## Plot Metrics

In [77]:
def plot_metrics(metrics_df, n_trajectories = 1, title = "", line_name_prefix="", line_name_subfix = [""]):
    fig_2d = go.Figure()
    
    
    for i in range(n_trajectories):
        subfix = lambda : line_name_subfix[i] if len(line_name_subfix) > 1 else i
        trajectory_name = line_name_prefix +": " + str(subfix())

        fig_2d.add_scatter( 
                    x = metrics_df.index,
                    y = metrics_df.distance, 
                    marker = dict(size=2),
                    line = dict(width=2+(i*0.1)),
                    mode = 'lines',
                    name = trajectory_name,
                    showlegend = True)
        
    fig_2d.update_layout(
            title_text = title,
            autosize = False,
            width = 800,
            height = 800)

    fig_2d.show()

time: 0 ns (started: 2022-09-22 02:48:23 -05:00)


# PRUEBAS

In [49]:
explorers = Explorers()

time: 0 ns (started: 2022-09-22 02:31:31 -05:00)


In [44]:
n_traj = 3

time: 0 ns (started: 2022-09-22 02:27:46 -05:00)


In [50]:
speeds = [3,6,6]

time: 0 ns (started: 2022-09-22 02:32:22 -05:00)


In [51]:
bm_2d_df = explorers.trace_trajectories(n_traj,"BM_2d", speeds= speeds )

time: 31 ms (started: 2022-09-22 02:32:24 -05:00)


In [52]:
plot_trajectories(bm_2d_df,n_traj,"Brownian Motion", "BM_speed_", speeds)

time: 16 ms (started: 2022-09-22 02:32:49 -05:00)


In [53]:
metric = TrajectoryMetrics()

time: 0 ns (started: 2022-09-22 02:33:22 -05:00)


In [61]:
aux_df = bm_2d_df.loc[bm_2d_df['traj']==0,['x','y']]

time: 0 ns (started: 2022-09-22 02:37:26 -05:00)


In [62]:
aux_df 

,x,y
0,0.000000,0.000000
1,2.374076,1.834057
2,0.453355,4.138581
3,-1.744502,6.180497
4,0.270272,8.403262
...,...,...
995,7.101961,51.685052
996,8.656177,54.251063
997,11.649058,54.457620
998,11.903334,51.468416


time: 15 ms (started: 2022-09-22 02:37:28 -05:00)


In [63]:
pl_df = metric.get_path_lengt(aux_df )

time: 47 ms (started: 2022-09-22 02:37:59 -05:00)


In [78]:
plot_metrics(pl_df, title = "Path lenght", line_name_prefix="Path lenght")

time: 16 ms (started: 2022-09-22 02:48:29 -05:00)
